In [ ]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Imports
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    confusion_matrix, roc_auc_score, roc_curve
)
import numpy as np
from collections import Counter

# ✅ Count and print number of images per class
def count_labels(dataset, class_names, split_name=""):
    labels = [sample[1] for sample in dataset]
    label_counts = Counter(labels)
    print(f"\U0001F4CA {split_name} split distribution:")
    for idx in range(len(class_names)):
        print(f"   \U0001F4C1 {class_names[idx]}: {label_counts[idx]} images")
    print()

# ✅ Load dataset
def load_food_dataset(base_path, img_size=224, batch_size=32, split_ratio=0.8):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

    full_dataset = datasets.ImageFolder(root=base_path, transform=transform)
    class_names = full_dataset.classes
    total_images = len(full_dataset)
    print(f"\U0001F4E6 Total images found: {total_images} across {len(class_names)} classes: {class_names}")

    train_size = int(split_ratio * total_images)
    val_size = total_images - train_size
    train_set, val_set = random_split(full_dataset, [train_size, val_size])

    print(f"\U0001F7E2 Training images: {len(train_set)}")
    count_labels(train_set, class_names, split_name="Training")

    print(f"\U0001F535 Validation images: {len(val_set)}")
    count_labels(val_set, class_names, split_name="Validation")

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, class_names

# ✅ Simple CNN
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256), nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

# ✅ Combined VGG19 + SimpleCNN refinement model
class VGG19_SimpleCNN(nn.Module):
    def __init__(self, num_classes, pretrained=True, feature_extract=True):
        super().__init__()
        vgg = models.vgg19_bn(pretrained=pretrained)  # ✅ FIXED
        self.features = vgg.features
        if feature_extract:
            for p in self.features.parameters():
                p.requires_grad = False

        self.refine = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(256, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.refine(x)
        return x

# ✅ Get model
def get_model(model_name, num_classes, device, pretrained=True, feature_extract=True):
    model_name = model_name.lower()
    if model_name == "simple":
        model = SimpleCNN(num_classes)
    elif model_name == "vgg19":
        model = models.vgg19_bn(pretrained=pretrained)
        if feature_extract:
            for p in model.features.parameters():
                p.requires_grad = False
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif model_name in {"combo", "vgg19+simple"}:
        model = VGG19_SimpleCNN(num_classes, pretrained, feature_extract)
    else:
        raise ValueError("Unknown model_name")
    return model.to(device)

# ✅ Train model
def evaluate_model(model, val_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return 100. * correct / total

def train_model(model, train_loader, val_loader, device, epochs=4, lr=0.001, save_path='model.pth'):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_acc_list, val_acc_list = [], []
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        correct, total, train_loss = 0, 0, 0.0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_acc = 100. * correct / total
        val_acc = evaluate_model(model, val_loader, device)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        print(f"Epoch {epoch+1}: Train Acc = {train_acc:.2f}%, Val Acc = {val_acc:.2f}%")

    torch.save(model.state_dict(), save_path)
    print(f"✅ Model saved at: {save_path}")

    plt.plot(train_acc_list, label='Train Accuracy')
    plt.plot(val_acc_list, label='Val Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training & Validation Accuracy')
    plt.grid(True)
    plt.legend()
    plt.show()

# ✅ Evaluate
def evaluate_model_detailed(model, val_loader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)

    probs, roc_data_available = None, False
    try:
        probs = []
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to(device)
                outputs = model(images)
                probs.append(torch.softmax(outputs, dim=1).cpu().numpy())
        probs = np.vstack(probs)
        from sklearn.preprocessing import label_binarize
        labels_one_hot = label_binarize(all_labels, classes=list(range(probs.shape[1])))
        auc = roc_auc_score(labels_one_hot, probs, average='weighted', multi_class='ovr')
        roc_data_available = True
    except:
        auc = None

    return accuracy, precision, recall, cm, auc, roc_data_available, all_labels, all_preds, probs

# ✅ Predict
def predict_image(model, image_path, class_names, device):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = output.max(1)
        predicted_class = class_names[predicted.item()]
    print(f"🔎 Prediction for '{os.path.basename(image_path)}': {predicted_class}")

# ✅ Main
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"⚙️ Using device: {device}")

    data_path = '/content/drive/MyDrive/Dataset/Dataset/Combined/Our dataset'
    model_save_path = '/content/drive/MyDrive/Dataset/Dataset/Samifood_model_vgg_ourdataset.pth'

    train_loader, val_loader, class_names = load_food_dataset(data_path)
    model = get_model("combo", len(class_names), device, pretrained=True, feature_extract=True)

    train_model(model, train_loader, val_loader, device, epochs=50, lr=1e-5, save_path=model_save_path)

    model.load_state_dict(torch.load(model_save_path, map_location=device))
    accuracy, precision, recall, cm, auc, roc_data_available, all_labels, all_preds, probs = evaluate_model_detailed(model, val_loader, device)
    print(f"\n📊 Validation Metrics:\nAccuracy: {accuracy:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}")
    print("Confusion Matrix:")
    print(cm)

    if auc is not None:
        print(f"ROC AUC: {auc:.4f}")
        plt.figure(figsize=(8,6))
        for i in range(len(class_names)):
            fpr, tpr, _ = roc_curve(np.array(all_labels) == i, probs[:, i])
            plt.plot(fpr, tpr, label=f"Class {class_names[i]} (AUC = {roc_auc_score(np.array(all_labels) == i, probs[:, i]):.2f})")
        plt.plot([0,1],[0,1],'k--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title("ROC Curve - Multiclass")
        plt.legend(loc='lower right')
        plt.grid(True)
        plt.show()
    else:
        print("ROC AUC curve not available.")

    test_image_path = f"/content/drive/MyDrive/Dataset/Dataset/Combined/Our dataset/Sugarcane Juice/f52981ca1c92d937fd3fd6fc9eb26215.jpg"
    predict_image(model, test_image_path, class_names, device)

# ✅ Run
if __name__ == "__main__":
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⚙️ Using device: cpu
📦 Total images found: 1475 across 10 classes: ['Fuchka', 'Jhalmuri', 'Kalavuna', 'Khichuri', 'Mishti', 'Pitha', 'Puri', 'Roshmalai', 'Shingara', 'Sugarcane Juice']
🟢 Training images: 1180
📊 Training split distribution:
   📁 Fuchka: 121 images
   📁 Jhalmuri: 122 images
   📁 Kalavuna: 131 images
   📁 Khichuri: 115 images
   📁 Mishti: 125 images
   📁 Pitha: 122 images
   📁 Puri: 119 images
   📁 Roshmalai: 86 images
   📁 Shingara: 122 images
   📁 Sugarcane Juice: 117 images

🔵 Validation images: 295
📊 Validation split distribution:
   📁 Fuchka: 31 images
   📁 Jhalmuri: 30 images
   📁 Kalavuna: 21 images
   📁 Khichuri: 37 images
   📁 Mishti: 27 images
   📁 Pitha: 30 images
   📁 Puri: 33 images
   📁 Roshmalai: 21 images
   📁 Shingara: 30 images
   📁 Sugarcane Juice: 35 images



Epoch 1/50 - Training: 100%|██████████| 37/37 [19:59<00:00, 32.43s/it]


Epoch 1: Train Acc = 13.73%, Val Acc = 16.61%


Epoch 2/50 - Training: 100%|██████████| 37/37 [19:30<00:00, 31.64s/it]


Epoch 2: Train Acc = 30.68%, Val Acc = 51.53%


...

Epoch 29/50 - Training: 100%|██████████| 37/37 [18:23<00:00, 29.81s/it]


Epoch 29: Train Acc = 90.93%, Val Acc = 89.83%


Epoch 30/50 - Training: 100%|██████████| 37/37 [18:34<00:00, 30.13s/it]


Epoch 30: Train Acc = 90.76%, Val Acc = 89.49%


Epoch 31/50 - Training:   0%|          | 0/37 [00:00<?, ?it/s]